In [ ]:
!pip install turicreate

In [ ]:
import os
import pandas as pd
import numpy as np
from zipfile import ZipFile
import turicreate

# Load Dataset

In [ ]:
ds_dir = '../input/coupon-purchase-prediction'

In [ ]:
#unzip dataset
with ZipFile(os.path.join(ds_dir,"coupon_detail_train.csv.zip"), 'r') as zipObj:
   zipObj.extractall()
with ZipFile(os.path.join(ds_dir,"coupon_list_test.csv.zip"), 'r') as zipObj:
   zipObj.extractall()
with ZipFile(os.path.join(ds_dir,"coupon_list_train.csv.zip"), 'r') as zipObj:
   zipObj.extractall()
with ZipFile(os.path.join(ds_dir,"coupon_visit_train.csv.zip"), 'r') as zipObj:
   zipObj.extractall()
with ZipFile(os.path.join(ds_dir,"sample_submission.csv.zip"), 'r') as zipObj:
   zipObj.extractall()
with ZipFile(os.path.join(ds_dir,"user_list.csv.zip"), 'r') as zipObj:
   zipObj.extractall()

In [ ]:
#Dataset
cd_train = pd.read_csv('coupon_detail_train.csv')
cl_test = pd.read_csv('coupon_list_test.csv')
cl_train = pd.read_csv('coupon_list_train.csv')
#cv_train = pd.read_csv('coupon_visit_train.csv')
#pref_loc = pd.read_csv(os.path.join(ds_dir,'prefecture_locations.csv'))
sample_sub = pd.read_csv('sample_submission.csv')
user_list = pd.read_csv('user_list.csv')

#Translator
pref = pd.read_csv(os.path.join(dsdir,'pref.csv'),delimiter=';',index_col='jpn')
pref_office = pd.read_csv(os.path.join(dsdir,'pref_office.csv'),delimiter=';',index_col='jpn')
small_area_name = pd.read_csv(os.path.join(dsdir,'small_area_name.csv'),delimiter=';',index_col='jpn')
big_area_name = pd.read_csv(os.path.join(dsdir,'big_area_name.csv'),delimiter=';',index_col='jpn')
capsule_text = pd.read_csv(os.path.join(dsdir,'capsule_text.csv'),delimiter=';',index_col='jpn')
genre_name = pd.read_csv(os.path.join(dsdir,'genre.csv'),delimiter=';',index_col='jpn')

## Translate JPN TO EN

#CAPSULE TEXT
cl_test.CAPSULE_TEXT = cl_test.CAPSULE_TEXT.replace(capsule_text.to_dict()['en'])
cl_train.CAPSULE_TEXT = cl_train.CAPSULE_TEXT.replace(capsule_text.to_dict()['en'])

#GENRE NAME
cl_test.GENRE_NAME = cl_test.GENRE_NAME.replace(genre_name.to_dict()['en'])
cl_train.GENRE_NAME = cl_train.GENRE_NAME.replace(genre_name.to_dict()['en'])

#PREF NAME
cl_test.ken_name = cl_test.ken_name.replace(pref.to_dict()['en'])
cl_train.ken_name = cl_train.ken_name.replace(pref.to_dict()['en'])
pref_loc.PREF_NAME = pref_loc.PREF_NAME.replace(pref.to_dict()['en'])
user_list.PREF_NAME = user_list.PREF_NAME.replace(pref.to_dict()['en'])

#PREFECTUAL_OFFICE
pref_loc.PREFECTUAL_OFFICE = pref_loc.PREFECTUAL_OFFICE.replace(pref_office.to_dict()['en'])

#SMALL_AREA_NAME
cd_train.SMALL_AREA_NAME = cd_train.SMALL_AREA_NAME.replace(small_area_name.to_dict()['en'])
cl_test.small_area_name = cl_test.small_area_name.replace(small_area_name.to_dict()['en'])
cl_train.small_area_name = cl_train.small_area_name.replace(small_area_name.to_dict()['en'])

#large_area_name
cl_test.large_area_name = cl_test.large_area_name.replace(big_area_name.to_dict()['en'])
cl_train.large_area_name = cl_train.large_area_name.replace(big_area_name.to_dict()['en'])

# Preprocessing and Convert Data to SFrame

In [ ]:
cd_train = cd_train.groupby(['USER_ID_hash', 'COUPON_ID_hash'])['PURCHASEID_hash'].count().to_frame().reset_index()

In [ ]:
cl_train.drop(['VALIDFROM','VALIDEND'],axis=1,inplace=True)
cl_test.drop(['VALIDFROM','VALIDEND'],axis=1,inplace=True)

#cl_train.VALIDPERIOD.fillna(cl_train.VALIDPERIOD.value_counts().index[0], inplace=True)
#cl_test.VALIDPERIOD.fillna(cl_test.VALIDPERIOD.value_counts().index[0], inplace=True)

cl_train.VALIDPERIOD.fillna(3650, inplace=True)
cl_test.VALIDPERIOD.fillna(3650, inplace=True)

cl_train.fillna(1, inplace=True)
cl_test.fillna(1, inplace=True)

In [ ]:
user_list.drop(['WITHDRAW_DATE'],axis=1,inplace=True)
user_list.PREF_NAME.fillna('Tokyo',inplace=True)

In [ ]:
#on create
observation_data = turicreate.SFrame(cd_train)

item_data = turicreate.SFrame(pd.concat([cl_train,cl_test]))

user_data = turicreate.SFrame(user_list)

In [ ]:
#on recommend
users = turicreate.SFrame(user_list[['USER_ID_hash']])

items = turicreate.SFrame(cl_test[['COUPON_ID_hash']])

# Fitting

In [ ]:
model = turicreate.recommender.ranking_factorization_recommender.create(solver='adagrad', observation_data=observation_data, user_id='USER_ID_hash', item_id='COUPON_ID_hash', target='PURCHASEID_hash', user_data=user_data, item_data=item_data, num_factors=128, random_seed=0, verbose=1)

# Recommend

In [ ]:
res = model.recommend(users, k=10, items=items, verbose=False).to_dataframe()

# Convert to Submission Format

In [ ]:
def clean_prediction(row):
    data = row.PURCHASED_COUPONS
    data = str("".join(str(data))[2:-2].replace("', '"," "))
    return data

In [ ]:
res = res.groupby('USER_ID_hash')['COUPON_ID_hash'].apply(list).reset_index(name='PURCHASED_COUPONS')

In [ ]:
res['PURCHASED_COUPONS'] = res.apply(clean_prediction, axis=1)

In [ ]:
res.to_csv('sub_cpp_turi_rfr_adagrad.csv', index=False)

In [ ]:
res